In [3]:
import pandas as pd
import numpy as np

from ckiptagger import WS, POS
from tqdm.notebook import tqdm

In [4]:
df_train = pd.read_csv('news_clustering_train.tsv', sep='\t')
train_titles = {row['index']: row['title'] for _, row in df_train.iterrows()}
train_classes = {row['index']: row['class'] for _, row in df_train.iterrows()}

df_test = pd.read_csv('news_clustering_test.tsv', sep='\t')
test_titles = {row['index']: row['title'] for _, row in df_test.iterrows()}
test_classes = {row['index']: row['class'] for _, row in df_test.iterrows()}

all_news_class = ['體育', '財經', '科技', '旅遊', '農業', '遊戲']

# 斷詞 + POS

In [7]:
# 忽略警告
import warnings
warnings.simplefilter("ignore")
ws, pos = WS('./data/'), POS('./data/')

In [8]:
train_title_cuts = {}
for index, title in tqdm(train_titles.items()):
    word_s = ws([title])
    word_p = pos(word_s)
    train_title_cuts[index] = list(zip(word_s[0], word_p[0]))

In [9]:
test_title_cuts = {}
for index, title in tqdm(test_titles.items()):
    word_s = ws([title])
    word_p = pos(word_s)
    test_title_cuts[index] = list(zip(word_s[0], word_p[0]))

# 尋找降維的詞向量：PPMI + SVD

In [10]:
word2index = {}
index2word = {}
unique_words = list(set(word for pairs in train_title_cuts.values() for word, _ in pairs))

for index, word in enumerate(unique_words):
    word2index[word] = index
    index2word[index] = word

In [11]:
len(word2index)

6690

如果使用one-hot就需要這麼大的維度的詞向量

In [12]:
# 建立Co-Matrix

vocab_size = len(word2index)
co_matrix = np.zeros(shape=(vocab_size, vocab_size), dtype=np.int32)

window_size = 1
# YOUR CODE HERE

# END YOUR CODE

In [13]:
co_matrix

array([[0, 2, 0, ..., 0, 0, 0],
       [2, 0, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

In [14]:
# 建立PPMI

def get_ppmi(co_matrix: np.ndarray, eps: float=1e-8):
    # YOUR CODE HERE

    # END YOUR CODE
    return M

ppmi = get_ppmi(co_matrix)

In [15]:
ppmi

array([[ 0.      ,  9.267157,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 9.267157,  0.      , 11.515084, ...,  0.      ,  0.      ,
         0.      ],
       [ 0.      , 11.515084,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       ...,
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ]], dtype=float32)

In [16]:
# 進行SVD分解，並得到降維的詞向量

from sklearn.decomposition import TruncatedSVD

# 使用`TruncatedSVD`進行降維，降維到dim=1000
# YOUR CODE HERE

# END YOUR CODE

In [17]:
word_vectors.shape

(6690, 1000)

# 新的詞向量 + Group mean vector: 測試

In [19]:
train_svd_vectors = {}

for index, pairs in train_title_cuts.items():
    selected_word_vectors = []
    
    for word, flag in pairs:
        if word in word2index and flag not in excluded_flags:
            selected_word_vectors.append(word_vectors[word2index[word], :])
    
    vector = np.sum(selected_word_vectors, axis=0)
    if np.sum(np.square(vector)) == 0:
        continue
    
    train_svd_vectors[index] = vector

In [20]:
test_svd_vectors = {}

for index, pairs in test_title_cuts.items():
    selected_word_vectors = []
    for word, flag in pairs:
        if word in word2index and flag not in excluded_flags:
            selected_word_vectors.append(word_vectors[word2index[word], :])
    vector = np.sum(selected_word_vectors, axis=0)
    if np.sum(np.square(vector)) == 0:
        continue
    test_svd_vectors[index] = vector

In [21]:
group_vectors = {news_class: [] for news_class in all_news_class}
group_mean_vector = {}

for index, pairs in sorted(train_title_cuts.items()):
    news_class = train_classes[index]
    group_vectors[news_class].append(vector)

for news_class, vectors in group_vectors.items():
    group_mean_vector[news_class] = np.mean(vectors, axis=0)
    
display(group_mean_vector)

In [22]:
def cosine_similarity(bow1, bow2):
    len_bow1 = np.sqrt(np.sum(np.square(bow1)))
    len_bow2 = np.sqrt(np.sum(np.square(bow2)))
    return np.sum(bow1 * bow2) / (len_bow1 * len_bow2)

In [23]:
classification = {news_class: [] for news_class in all_news_class}
for index, vector in sorted(test_svd_vectors.items()):
    if np.sum(np.square(vector)) == 0:
        continue

    max_val = -2.0
    max_class = None
    for news_class, ref_vector in group_mean_vector.items():
        val = cosine_similarity(ref_vector, vector)
        if val > max_val:
            max_class = news_class
            max_val = val

    classification[max_class].append(index)

In [24]:
from collections import Counter

accuracy=[]

for group, ids in classification.items():
    counter = Counter([test_classes[id] for id in ids])
    prediciton=round(counter[group]/sum(counter.values())*100,2)
    accuracy.append(prediciton)
    print(f'{group} : {str(counter):70} \taccuracy : {prediciton}%')

print(f'\nAverage accuracy : {round(np.mean(accuracy),2)}%')

predict 體育 :  Counter({'體育': 61, '遊戲': 10, '旅遊': 9, '財經': 8, '科技': 5, '農業': 4})
predict 財經 :  Counter({'財經': 62, '科技': 25, '農業': 16, '體育': 8, '遊戲': 8, '旅遊': 7})
predict 科技 :  Counter({'科技': 51, '體育': 15, '財經': 14, '農業': 9, '遊戲': 9, '旅遊': 8})
predict 旅遊 :  Counter({'旅遊': 58, '農業': 11, '科技': 5, '財經': 4, '體育': 2, '遊戲': 2})
predict 農業 :  Counter({'農業': 58, '旅遊': 7, '體育': 4, '財經': 4, '遊戲': 4, '科技': 2})
predict 遊戲 :  Counter({'遊戲': 67, '科技': 11, '旅遊': 9, '體育': 8, '財經': 8, '農業': 1})
